# Idling

## Description
When agents do not perform work during work time, but rather focus on non-work related activities or spend time socializing with their colleagues.

In [16]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [17]:
log_name = 'propr_test'

fn_logs = {
    'propr': '../data/preproc/proprietary.csv',
    'propr_ref': '../data/preproc/propr_train-70.csv',
    'propr_test': '../data/preproc/propr_test-30.csv'
}

In [18]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp
0,Precheck,ID-2,6/05/2011 16:09,6/05/2011 16:27,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00
1,Application_Processing_Branches,ID-2,9/05/2011 11:14,9/05/2011 11:16,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00
2,Precheck,ID-2,9/05/2011 15:44,9/05/2011 15:58,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00
3,Processing_Incomplete_Orders,ID-2,9/05/2011 16:10,9/05/2011 16:10,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00
4,Application_Processing_Branches,ID-2,10/05/2011 8:52,10/05/2011 9:32,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,ID-1720,20/05/2011 17:24,20/05/2011 17:26,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00
5286,Check_of_Processing_Applications,ID-1720,23/05/2011 15:53,23/05/2011 16:06,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00
5287,Processing_of_Applications,ID-1720,26/05/2011 9:25,26/05/2011 9:25,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00
5288,Archieving,ID-1720,26/05/2011 10:38,26/05/2011 10:47,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00


In [19]:
# encode activity labels as short names for presentation purpose

print('There exist {} unique activity labels'.format(
    log['activity'].nunique()
))

# use initial letters for encoding
act_labels_short = dict()
for label in log['activity'].unique():
    label_short = ''.join([w[0] for w in label.split('_')])
    act_labels_short[label] = label_short
print('There are {} short labels after encoding'.format(
    len(act_labels_short.values())
))
for k in sorted(act_labels_short.keys()):
    print('{:>50} -> {:>30}'.format(k, act_labels_short[k]))

log['activity_short'] = log['activity'].apply(
    lambda x: act_labels_short[x]
)

# add activity duration
log['start timestamp'] = pd.to_datetime(log['start timestamp'], format='mixed', dayfirst=True)
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], 
format='mixed', dayfirst=True)
# duration measured in minutes
log['activity duration'] = (
    log['complete timestamp'] - log['start timestamp']
).dt.total_seconds() / 60

log['activity'] = log['activity_short']

log

There exist 25 unique activity labels
There are 25 short labels after encoding
                              Administrative_Tasks ->                             AT
                   Application_Processing_Branches ->                            APB
                                          Approval ->                              A
                                   Approval_Branch ->                             AB
                          Approval_Executive_Board ->                            AEB
                                        Archieving ->                              A
                                 Check_of_Approval ->                            CoA
                                Check_of_Documents ->                            CoD
                  Check_of_Processing_Applications ->                           CoPA
                                               End ->                              E
                                   Further_inquiry ->                  

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_short,activity duration
0,P,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,P,18.0
1,APB,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,APB,2.0
2,P,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,P,14.0
3,PIO,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,PIO,0.0
4,APB,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,APB,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,P,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,P,2.0
5286,CoPA,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,CoPA,13.0
5287,PoA,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,PoA,0.0
5288,A,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,A,9.0


In [20]:
# proprietary
log['department'] = log['resource'].apply(
    lambda x: x.split('-')[0]
)
log['department_role'] = log['resource'].apply(
    lambda x: x.split('-')[0] + '-' + x.split('-')[1]
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_short,activity duration,department,department_role
0,P,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,P,18.0,090,090-10
1,APB,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,APB,2.0,010,010-23
2,P,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,P,14.0,090,090-10
3,PIO,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,PIO,0.0,010,010-23
4,APB,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,APB,40.0,010,010-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,P,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,P,2.0,000,000-3
5286,CoPA,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,CoPA,13.0,000,000-2
5287,PoA,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,PoA,0.0,010,010-23
5288,A,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,A,9.0,010,010-23


In [21]:
sel_resources = log.loc[log['department_role'] == '010-24', 'resource'].unique()
print(len(sel_resources))

11


## Social-Idling-1

In [22]:
log = log.sort_values(by='start timestamp')
log['start timestamp'] = pd.to_datetime(log['start timestamp'], format='mixed', dayfirst=True)
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], format='mixed', dayfirst=True)


### Calculate waiting time between activity instances for each resource

In [23]:
df_res_waiting = list()
for res, events in log.groupby('resource'):
    for i in range(len(events) - 1):
        pre_event = events.iloc[i]
        next_event = events.iloc[i+1]
        waiting_time = (
            next_event['start timestamp'] - 
            pre_event['complete timestamp']
        )
        df_res_waiting.append({
            'resource': res,
            'preceding case_id': pre_event['case_id'],
            'preceding activity': pre_event['activity'],
            'preceding activity completion': pre_event['complete timestamp'],
            'preceding activity completion date': pre_event['complete timestamp'].date(),
            'next case_id': next_event['case_id'],
            'next activity': next_event['activity'],
            'next activity start': next_event['start timestamp'],
            'next activity start date': next_event['start timestamp'].date(),
            'activity_pair': pre_event['activity'] + '->' + next_event['activity'],
            'waiting time': waiting_time
        })
df_res_waiting = pd.DataFrame(df_res_waiting)
df_res_waiting['waiting time'] =  df_res_waiting['waiting time'].dt.total_seconds() / 60
# NOTE: only non-negative waiting time makes sense
df_res_waiting = df_res_waiting[df_res_waiting['waiting time'] >= 0]
# NOTE: only consider waiting on the same date
df_res_waiting = df_res_waiting[
    df_res_waiting['preceding activity completion date'] == 
    df_res_waiting['next activity start date']
]

df_res_waiting = df_res_waiting[df_res_waiting['resource'].isin(sel_resources)]
df_res_waiting

,resource,preceding case_id,preceding activity,preceding activity completion,preceding activity completion date,next case_id,next activity,next activity start,next activity start date,activity_pair,waiting time
2101,010-24-01,ID-579,PoA,2011-03-14 12:29:00,2011-03-14,ID-579,PoA,2011-03-14 12:29:00,2011-03-14,PoA->PoA,0.0
2102,010-24-01,ID-579,PoA,2011-03-14 13:06:00,2011-03-14,ID-579,GSoC,2011-03-14 14:00:00,2011-03-14,PoA->GSoC,54.0
2104,010-24-01,ID-621,PoA,2011-03-17 14:52:00,2011-03-17,ID-621,GSoC,2011-03-17 14:52:00,2011-03-17,PoA->GSoC,0.0
2105,010-24-01,ID-621,GSoC,2011-03-17 16:05:00,2011-03-17,ID-75,PoA,2011-03-17 16:07:00,2011-03-17,GSoC->PoA,2.0
2106,010-24-01,ID-75,PoA,2011-03-17 16:07:00,2011-03-17,ID-75,GSoC,2011-03-17 16:07:00,2011-03-17,PoA->GSoC,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3703,010-24-26,ID-1371,CoPA,2011-05-30 11:56:00,2011-05-30,ID-946,CoPA,2011-05-30 15:05:00,2011-05-30,CoPA->CoPA,189.0
3704,010-24-26,ID-946,CoPA,2011-05-30 15:06:00,2011-05-30,ID-1466,PSP,2011-05-30 15:23:00,2011-05-30,CoPA->PSP,17.0
3706,010-24-26,ID-1466,PoAS,2011-05-31 07:35:00,2011-05-31,ID-547,PIO,2011-05-31 09:32:00,2011-05-31,PoAS->PIO,117.0
3707,010-24-26,ID-547,PIO,2011-05-31 09:32:00,2011-05-31,ID-547,CoPA,2011-05-31 15:04:00,2011-05-31,PIO->CoPA,332.0


### Identify resources with longer waiting time

In [24]:
df_res_waiting_q90 = list()
for res, events in df_res_waiting.groupby('resource'):
    df_res_waiting_q90.append({
        'resource': res,
        'waiting_time_q90': events['waiting time'].quantile(0.90)
    })
df_res_waiting_q90 = pd.DataFrame(df_res_waiting_q90)

In [25]:
# waiting time
# df_res_waiting = df_res_waiting[df_res_waiting['waiting time'] <= 60 * 8]

alt.vconcat(
    alt.Chart(
        df_res_waiting,
    ).mark_boxplot().encode(
        y=alt.Y('resource:O').title('resource ID'),
        x=alt.X('waiting time:Q').title('waiting time (minutes)'),
    ).properties(height=150),
    # alt.Chart(
    #     df_res_waiting_q90,
    # ).mark_bar().encode(
    #     y=alt.Y('resource:O'),
    #     x=alt.X('waiting_time_q90:Q').title('90th percentile of waiting time (minutes)'),
    #     tooltip=['waiting_time_q90']
    # ),
    # alt.layer(
    #     alt.Chart(
    #         df_res_waiting,
    #     ).mark_rect().encode(
    #         y=alt.Y('resource:O').title('resource ID'),
    #         # x=alt.X('preceding activity:N').title('previous activity performed'),
    #         x=alt.X('next activity:N').title('next activity to perform'),
    #         # x=alt.X('activity_pair:N'),
    #         color=alt.Color('mean(waiting time):Q').scale(scheme='lightgreyred').title(['average waiting', 'time (minutes)'])
    #     ),
    #     alt.Chart(
    #         df_res_waiting
    #     ).mark_text(baseline='middle').encode(
    #         y=alt.Y('resource:O').title('resource ID'),
    #         # x=alt.X('preceding activity:N').title('previous activity performed'),
    #         x=alt.X('next activity:N').title('next activity to perform'),
    #         # x=alt.X('activity_pair:N'),
    #         text=alt.Text('mean(waiting time):Q', format='.0f')
    #     )
    # )
).resolve_scale(color='independent')

alt.VConcatChart(...)

In [26]:
# test if there are differences in waiting time per resource
# df_res_waiting = df_res_waiting.dropna(how='any')
from scipy.stats import kruskal
wt_per_resource = [
    df_res_waiting.loc[df_res_waiting['resource'] == r, 'waiting time'].to_numpy()
    for r in df_res_waiting['resource'].unique()
]
for x in wt_per_resource:
    print(len(x))
kruskal(*wt_per_resource, nan_policy='raise')
# if p < 0.05, reject the null hypothesis that the underlying population median
# of all groups are equal

# Result: p << 0.01
# i.e., at 1% significance level, the differences are significant

272
97
73
4
22
45
115
133
454


KruskalResult(statistic=85.35405582370997, pvalue=4.06352650081096e-15)

In [27]:
# pariwise check
# Mann Whitney U
# H0 (null): samples are drawn from an identical distribution
# Ha: one distribution shifted either to the left or to the right of the other distribution
from itertools import combinations
from scipy.stats import mannwhitneyu

for x, y in combinations(sorted(df_res_waiting['resource'].unique()), r=2):
    print('Wilcoxon Rank Sum Test for {} and {}'.format(x, y))
    arr_x = df_res_waiting.loc[df_res_waiting['resource'] == x, 'waiting time'].to_numpy()
    arr_y = df_res_waiting.loc[df_res_waiting['resource'] == y, 'waiting time'].to_numpy()
    test_result = mannwhitneyu(arr_x, arr_y)
    if test_result.pvalue < 0.05:
        # reject H0 in favor of Ha
        print('\t{}'.format(mannwhitneyu(arr_x, arr_y)))

Wilcoxon Rank Sum Test for 010-24-01 and 010-24-02
	MannwhitneyuResult(statistic=15548.5, pvalue=0.007265744609404486)
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-04
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-11
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-12
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-13
	MannwhitneyuResult(statistic=4007.0, pvalue=0.00016297590066838586)
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-17
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-23
Wilcoxon Rank Sum Test for 010-24-01 and 010-24-26
	MannwhitneyuResult(statistic=45860.0, pvalue=5.192613525315917e-09)
Wilcoxon Rank Sum Test for 010-24-02 and 010-24-04
	MannwhitneyuResult(statistic=2821.0, pvalue=0.01867153685904309)
Wilcoxon Rank Sum Test for 010-24-02 and 010-24-11
Wilcoxon Rank Sum Test for 010-24-02 and 010-24-12
Wilcoxon Rank Sum Test for 010-24-02 and 010-24-13
	MannwhitneyuResult(statistic=1101.0, pvalue=1.1067481381457668e-06)
Wilcoxon Rank Sum Test for 010-24-02 and 010-24

In [28]:
# resource 010-24-13/17/26
# are they significantly higher in waiting time

others = [x for x in df_res_waiting['resource'].unique() if x not in 
          {'010-24-13', '010-24-17', '010-24-26'}]

In [29]:
# if p < 0.05, reject null hypothesis in favor of the alternative:
# selected resources having greater waiting time
mannwhitneyu(
    df_res_waiting.loc[df_res_waiting['resource'] == '010-24-13', 'waiting time'].to_numpy(),
    df_res_waiting.loc[df_res_waiting['resource'].isin(others), 'waiting time'].to_numpy(),
    alternative='greater'
)

MannwhitneyuResult(statistic=18564.5, pvalue=9.567627650374726e-06)

In [30]:
# if p < 0.05, reject null hypothesis in favor of the alternative:
# selected resources having greater waiting time
mannwhitneyu(
    df_res_waiting.loc[df_res_waiting['resource'] == '010-24-17', 'waiting time'].to_numpy(),
    df_res_waiting.loc[df_res_waiting['resource'].isin(others), 'waiting time'].to_numpy(),
    alternative='greater'
)

MannwhitneyuResult(statistic=39210.0, pvalue=0.009578816135519261)

In [31]:
# if p < 0.05, reject null hypothesis in favor of the alternative:
# selected resources having greater waiting time
mannwhitneyu(
    df_res_waiting.loc[df_res_waiting['resource'] == '010-24-26', 'waiting time'].to_numpy(),
    df_res_waiting.loc[df_res_waiting['resource'].isin(others), 'waiting time'].to_numpy(),
    alternative='greater'
)

MannwhitneyuResult(statistic=175241.5, pvalue=5.832743293748254e-16)